# Create Delta Tables from Hive DDL

This notebook converts Hive/Impala DDL statements to Databricks Delta tables.

**What it does:**
- Reads Hive DDL files (.sql)
- Creates managed Delta tables in Databricks
- Preserves table structure, partitioning, and column types
- Creates empty tables (structure only, no data)

## Setup

In [ ]:
import sys
import os

# Get notebook directory and add to path
notebook_path = dbutils.notebook.entry_point.getDbutils().notebook().getContext().notebookPath().get()
sys.path.append(os.path.dirname(notebook_path))

from create_delta_tables import create_tables_from_hive_ddl

## Configuration

**Update these values:**

In [ ]:
# Target catalog and schema
CATALOG = "your_catalog"  # UPDATE
SCHEMA = "your_schema"    # UPDATE

## Single File

Create one table from a single DDL file

In [ ]:
# Path to your Hive DDL file
FILE_PATH = "/Volumes/catalog/schema/path/table.sql"  # UPDATE

result = create_tables_from_hive_ddl(
    input_file=FILE_PATH,
    catalog=CATALOG,
    schema=SCHEMA
)

print(f"✓ Created {result['success_count']} table(s)")

## Batch Process

Create multiple tables from a directory of DDL files

In [ ]:
# Path to directory containing .sql files
DIRECTORY = "/Volumes/catalog/schema/path/hive_ddls/"  # UPDATE

result = create_tables_from_hive_ddl(
    input_dir=DIRECTORY,
    catalog=CATALOG,
    schema=SCHEMA
)

print(f"✓ Created {result['success_count']} table(s)")
print(f"✗ Failed {result['fail_count']} table(s)")

## Verify

Check created tables

In [ ]:
%sql
SHOW TABLES IN your_catalog.your_schema;  -- UPDATE

## Notes

- Tables are created **empty** (structure only)
- Column types are preserved as-is from Hive
- Tables are managed Delta tables with auto-optimization
- To preview DDL before creating, add `dry_run=True` parameter